In [1]:
pip install openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.0/337.0 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 1.9 MB/s eta 0:00:00


In [2]:
from openai import OpenAI

client = OpenAI(
    api_key = "sk-proj-o15ahTYqSPi8F172r228T3BlbkFJciUyjaWf5FHyzdVRA4yF",
)

In [4]:
from google.colab import drive
drive.mount('/content/drive')

training_file_name = '/content/drive/My Drive/03. path/Finetune_Gpt-3.5-turbo/training_data.jsonl'
validation_file_name = '/content/drive/My Drive/03. path/Finetune_Gpt-3.5-turbo/validation_data.jsonl'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
training_file_id = client.files.create(
  file = open(training_file_name, "rb"),
  purpose = "fine-tune"
)

validation_file_id = client.files.create(
  file = open(validation_file_name, "rb"),
  purpose = "fine-tune"
)

print(f"Training File ID: {training_file_id}")
print(f"Validation File ID: {validation_file_id}")

Training File ID: FileObject(id='file-rLfsk3CrBmZ0uvwqe1cLu1QS', bytes=5598, created_at=1722160911, filename='training_data.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)
Validation File ID: FileObject(id='file-aTiMJukivOKqeizliT0bN9S3', bytes=2792, created_at=1722160912, filename='validation_data.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)


In [7]:
response = client.fine_tuning.jobs.create(
  training_file=training_file_id.id,
  validation_file=validation_file_id.id,
  model="gpt-3.5-turbo",
  hyperparameters={
    "n_epochs": 15,
	  "batch_size": 3,
	  "learning_rate_multiplier": 0.3
  }
)
job_id = response.id
status = response.status

print(f'Fine-tunning model with jobID: {job_id}.')
print(f"Training Response: {response}")
print(f"Training Status: {status}")

Fine-tunning model with jobID: ftjob-TQ8TYveVMVxO2rvHc4ddAHos.
Training Response: FineTuningJob(id='ftjob-TQ8TYveVMVxO2rvHc4ddAHos', created_at=1722161363, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs=15, batch_size=3, learning_rate_multiplier=0.3), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-5c13hZRgHBn8Les2z6Q4Zx9w', result_files=[], seed=157131863, status='validating_files', trained_tokens=None, training_file='file-rLfsk3CrBmZ0uvwqe1cLu1QS', validation_file='file-aTiMJukivOKqeizliT0bN9S3', estimated_finish=None, integrations=[], user_provided_suffix=None)
Training Status: validating_files


In [8]:
import signal
import datetime


def signal_handler(sig, frame):
    status = client.fine_tuning.jobs.retrieve(job_id).status
    print(f"Stream interrupted. Job is still {status}.")
    return

print(f"Streaming events for the fine-tuning job: {job_id}")

signal.signal(signal.SIGINT, signal_handler)

events = client.fine_tuning.jobs.list_events(fine_tuning_job_id=job_id)
try:
    for event in events:
        print(
            f'{datetime.datetime.fromtimestamp(event.created_at)} {event.message}'
        )
except Exception:
    print("Stream interrupted (client disconnected).")

Streaming events for the fine-tuning job: ftjob-TQ8TYveVMVxO2rvHc4ddAHos
2024-07-28 10:15:48 Step 51/55: training loss=0.68, validation loss=0.22, full validation loss=0.28
2024-07-28 10:15:43 Step 50/55: training loss=0.63, validation loss=0.33
2024-07-28 10:15:40 Step 49/55: training loss=0.50, validation loss=0.25
2024-07-28 10:15:35 Step 48/55: training loss=0.57, validation loss=0.33, full validation loss=0.28
2024-07-28 10:15:31 Step 47/55: training loss=0.65, validation loss=0.32
2024-07-28 10:15:28 Step 46/55: training loss=0.65, validation loss=0.24
2024-07-28 10:15:23 Step 45/55: training loss=0.52, validation loss=0.32, full validation loss=0.28
2024-07-28 10:15:19 Step 44/55: training loss=0.61, validation loss=0.22
2024-07-28 10:15:14 Step 43/55: training loss=0.51, validation loss=0.33
2024-07-28 10:15:12 Step 42/55: training loss=0.68, validation loss=0.25, full validation loss=0.28
2024-07-28 10:15:07 Step 41/55: training loss=0.49, validation loss=0.33
2024-07-28 10:15

In [9]:
import time

status = client.fine_tuning.jobs.retrieve(job_id).status
if status not in ["succeeded", "failed"]:
    print(f"Job not in terminal status: {status}. Waiting.")
    while status not in ["succeeded", "failed"]:
        time.sleep(2)
        status = client.fine_tuning.jobs.retrieve(job_id).status
        print(f"Status: {status}")
else:
    print(f"Finetune job {job_id} finished with status: {status}")

Finetune job ftjob-TQ8TYveVMVxO2rvHc4ddAHos finished with status: succeeded


In [16]:
result = client.fine_tuning.jobs.list()
print(f"Found {len(result.data)} finetune jobs.")

Found 11 finetune jobs.


In [17]:
fine_tuned_model = result.data[0].fine_tuned_model

In [18]:
print(fine_tuned_model)

ft:gpt-3.5-turbo-0125:personal::9pvEdjjn


In [19]:
answer = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "user", "content": "ما هي سياسة إرجاع الكتب لديكم؟"},
    ]
)

print(answer.choices[0].message)

ChatCompletionMessage(content='يعتمد سياسة إرجاع الكتب لدينا على القوانين المحلية وشروط البيع. عادة ما يكون لدينا سياسة إرجاع مرنة تسمح للعملاء بإعادة الكتب غير المستخدمة في حالة جديدة وغير تالفة في غضون فترة زمنية معينة بعد الشراء. يرجى الاتصال بنا للحصول على مزيد من التفاصيل حول سياسة الإرجاع الخاصة بنا.', role='assistant', function_call=None, tool_calls=None)


In [24]:
answer = client.chat.completions.create(
    model=fine_tuned_model,
    messages=[
        {"role": "user", "content": "ما هي سياسة إرجاع الكتب لديكم؟"},
    ]
)

print(answer.choices[0].message)

ChatCompletionMessage(content='نعم، زبائننا يحق لهم إرجاع الكتب حتى 30 يوماً بعد شراء الكتاب. إذا كان الزبون غير مرتاح بالكتاب الذي اشتراه لدينا، يمكنهم إرجاعه واسترداد قيمة الشراء بالكامل.', role='assistant', function_call=None, tool_calls=None)


In [25]:
answer = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "user", "content": "هل يمكنك أن توصي بكتاب جيد للأطفال من سن 8 إلى 10 سنوات"},
    ]
)

print(answer.choices[0].message)

ChatCompletionMessage(content='نعم بالتأكيد، إليك بعض الكتب الجيدة التي يمكن أن تناسب الأطفال في هذه الفئة العمرية:\n\n1. "هاري بوتر وحجر الفيلسوف" للكاتبة ج. ك. رولينغ: سلسلة مشهورة ومحببة للأطفال، تتحدث عن مغامرات هاري بوتر الشهيرة في عالم السحر.\n\n2. "العجمي" للكاتبة جودي بلوم: قصة عن صبي يكتسب أصدقاء جدد ويتغلب على تحديات الصبيان الشرسة في المدرسة الإبتدائية.\n\n3. "روالد دال ومصنع الشوكولا" للكاتب روالد دال: قصة مذهلة تروي قصة صبي يدعى تشارلي بوكيت يفوز بجولة في المصنع الشهير للشوكولا.\n\n4. "آن من ألوذ" للكاتبة لاي. إم. مونتغمري: قصة مؤثرة عن فتاة يتيمة تسمى آن شيرلي، تعيش مغامرات مليئة بالفكاهة والعجائب.\n\n5. "المدرسة المرعبة" للكاتبة إنيد بليتون: قصة مشوقة تدور حول فتاة تكتشف أسراراً مرعبة في مدرستها الجديدة.\n\nإن هذه الكتب تعتبر خيارات رائعة للأطفال من سن 8 إلى 10 سنوات، وقد تستمتعون بقراءتها معًا والنقاش حول القصص والشخصيات فيها.', role='assistant', function_call=None, tool_calls=None)


In [27]:
answer = client.chat.completions.create(
    model=fine_tuned_model,
    messages=[
        {"role": "user", "content": "هل يمكنك أن توصي بكتاب جيد للأطفال من سن 8 إلى 10 سنوات؟"},
    ]
)

print(answer.choices[0].message)

ChatCompletionMessage(content='بالتأكيد! من الكتب الجيدة لهذه المجموعة العمرية هو "هاري بوتر" لج. ك. رولينج. يحكي المسلسل قصة صبي ساحر يدخل إلى عالم سحري ويقاتل قوى الشر.', role='assistant', function_call=None, tool_calls=None)
